# Computing gradient

Training DNNs requires computing gradient of the loss with respect to the parameters, and this is topic of this notebook.

Your task will be to:

* Implement Evolution Search training of DNNs that does not require backpropagation
* Implement all forward and backward functions for a MLP with custom activation function

Goal is to:

* Introduce ingredients necessary to train NN:
    * model
    * loss
    * optimizer
* Introduce different ways of computing or estimating $\frac{\partial{L(w)}}{{\partial w}}$
* Get understanding of Evolution Search and Backpropagation algorithms

Exam:

* For exam you are expected to understand how backpropagation works. Questions might refer to implementation details in PyTorch. See also Resources section for a good video about backpropgation, and how it is implemented in PyTorch.

What's (probably) next:

* Go through each component of the training loop:
    * (Lab 5) Neural Networks practical, part 1: architecture, understanding what is learned
    * (Lab 6, ?) Neural Networks practical part 2: setting up data and loss
    * (Lab 7, ?) Neural Networks practical part 3: optimization
* (Lab 8, ?) On gradients's properties (e.g. adversarial examples, vanishing/exploding gradient, variance)

Resources:

* Backprop with focus on PyTorch: https://www.youtube.com/watch?v=ma2KXWblllc (see also other lectures from this series)

* Evolution Search https://eng.uber.com/deep-neuroevolution/, https://arxiv.org/abs/1712.06564

# SGD requires $\frac{\partial L (w) }{\partial w}$!

<p>
<font size=5>
$$\frac{\partial L (w) }{\partial w} = \frac{1}{K} \sum_{i=1}^{K} \frac{\partial L (x_i, w) }{\partial w}$$
</font>
</p>

<img width=600 src="https://github.com/gmum/nn2018/raw/master/lab/fig/3/sgd.png">

# Setup 

In [99]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tqdm
import json

import torch
import torch.nn.functional as F

from torch import optim
from torch import nn
from torch.autograd import Variable

from keras.datasets import fashion_mnist
from keras.utils import np_utils

%matplotlib inline
import matplotlib.pylab as plt
import matplotlib as mpl

mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['figure.figsize'] = (7, 7)
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['axes.labelsize'] = 12

# Get FashionMNIST (see 1b_FMNIST.ipynb for data exploration)
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Logistic regression needs 2D data
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# 0-1 normalization
x_train = x_train / 255.
x_test = x_test / 255.

# Convert to Torch Tensor. Just to avoid boilerplate code later
x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

# Use only first 1k examples. Just for notebook to run faster
x_train, y_train = x_train[0:1000], y_train[0:1000]
x_test, y_test = x_test[0:1000], y_test[0:1000]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Training loop in PyTorch

In this section we walk through implementation of traditional training loop  in PyTorch.

Necessary ingredients train NN:
    * model
    * loss
    * optimizer
    
<img width=600 src="https://github.com/gmum/nn2018/raw/master/lab/fig/4/smoothie.png">

Ref:

https://github.com/vinhkhuc/PyTorch-Mini-Tutorials/blob/master/3_neural_net.py

In [151]:
def build_model(input_dim, output_dim, nonlinearity_module=torch.nn.Sigmoid, weight_module=torch.nn.Linear):
    model = torch.nn.Sequential()
    model.add_module("linear_1", weight_module(input_dim, 512, bias=False))
    model.add_module("sigmoid_1", nonlinearity_module())
    model.add_module("linear_2", weight_module(512, output_dim, bias=False))
    return model

In [150]:
def sgd_step(model, loss, x_val, y_val, lr=0.1, momentum=0.9):
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    
    x = Variable(x_val, requires_grad=False)
    y = Variable(y_val, requires_grad=False)

    # Reset gradient
    optimizer.zero_grad()

    # Forward
    fx = model.forward(x)
    output = loss.forward(fx, y)

    # Backward
    output.backward()

    # Update parameters
    optimizer.step()

    return output.data[0]

In [102]:
def predict(model, x_val):
    x = Variable(x_val, requires_grad=False)
    output = model.forward(x)
    return output.data.numpy().argmax(axis=1)

In [103]:
def train(model, loss, step_fnc):
    torch.manual_seed(42)
    n_examples, n_features = x_train.size()
    n_classes = 10
    batch_size = 100

    for i in tqdm.tqdm(range(100), total=100):
        cost = 0.
        num_batches = n_examples // batch_size
        for k in range(num_batches):
            start, end = k * batch_size, (k + 1) * batch_size
            cost += sgd_step(model, loss, x_train[start:end], y_train[start:end])
        predY = predict(model, x_test)
        print(("Epoch %d, cost = %f, acc = %.2f%%"
              % (i + 1, cost / num_batches, 100. * np.mean(predY == y_test.numpy()))))
        
    return np.mean(predY == y_test.numpy())

## Run training

In [98]:
model = build_model(n_features, n_classes)
loss = torch.nn.CrossEntropyLoss(size_average=True)
train(step_fnc=sgd_step, loss=loss, model=model)

  1%|          | 1/100 [00:00<00:10,  9.19it/s]

Epoch 1, cost = 2.324177, acc = 9.70%
Epoch 2, cost = 2.293060, acc = 9.50%

  2%|▏         | 2/100 [00:00<00:10,  9.32it/s]


Epoch 3, cost = 2.282423, acc = 11.50%
Epoch 4, cost = 2.272841, acc = 14.40%

  4%|▍         | 4/100 [00:00<00:10,  9.25it/s]

...

 97%|█████████▋| 97/100 [00:08<00:00, 12.56it/s]


Epoch 98, cost = 1.316578, acc = 60.70%
Epoch 99, cost = 1.310077, acc = 61.00%

 99%|█████████▉| 99/100 [00:08<00:00, 12.66it/s]


Epoch 100, cost = 1.303664, acc = 61.10%

0.611

# Estimating $\frac{\partial L (w) }{\partial w}$

In this section we will implement different gradient computation functions

## Finite Difference
<p>
<font size=4>
Assuming we know direction we want to optimize in, we can approximate derivative simply by calculating

$$\langle \frac{\partial L (w) }{\partial w}, \vec{v} \rangle \approx \frac{L(w) - L(w + v)}{|\vec{v}|}$$


, where $L(w)$ denotes esimation of loss (e.g. over whole training set, over $5\%$ etc.). 

</font>
</p>

## Evolution Search

<p>
Intuitively, Evolution Search (ES) approximates gradient by computing multiple times finite difference. Importantly it works for non-differentiable costs as well.
</p>

<p>
<font size=4>
$$ \frac{\partial L (w) }{\partial w} \approx \frac{1}{N \sigma^2} \sum \vec{v} L(w + \vec{v})$$


, where $\epsilon$ is a normally distributed vector sampled from gauss of standard deviation $\sigma$, and $L(w)$ denotes esimation of loss (e.g. over whole training set, over $5\%$ etc.). 

</font>
</p>

<p>
<font size=4>
Note: while it is recommended to not use minibatching in ES, for simplicity we will.
</font>
</p>

In [137]:
def get_model_weights(model):
    params = model.state_dict()
    p_order = [p[0] for p in model.named_parameters()]
    return np.concatenate([params[w].cpu().numpy().reshape(-1, ) for w in p_order])
    
def set_model_weights(model, w):
    params = model.state_dict()  
    p_order = [p[0] for p in model.named_parameters()]
    id = 0
    for p in p_order:
        shape = params[p].shape
        D = np.prod(shape)
        params[p].copy_(torch.from_numpy(w[id:id + D].reshape(shape)))
        id += D
        
def L(w, loss, batch_size=10, x_tr=x_train, y_tr=y_train):
    set_model_weights(model, w)
    cost = 0.
    n_examples, n_features = x_tr.size()
    num_batches = n_examples // batch_size
    for k in range(num_batches): 
        start, end = k * batch_size, (k + 1) * batch_size
        x = Variable(x_tr[start:end], requires_grad=False)
        y = Variable(y_tr[start:end], requires_grad=False)
        fx = model.forward(x)
        cost += loss.forward(fx, y)
    return cost.data.numpy()[0] / num_batches

In [126]:
def sgd_step(model, loss, x_val, y_val, lr, momentum):
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    x = Variable(x_val, requires_grad=False)
    y = Variable(y_val, requires_grad=False)

    # Reset gradient
    optimizer.zero_grad()

    # Forward
    fx = model.forward(x)
    output = loss.forward(fx, y)

    # Backward
    output.backward()

    # Update parameters
    optimizer.step()

    return output.data[0]

In [148]:
def ES_grad(model, loss, x_tr, y_tr, sigma=0.001, N=100):
    grad = 0
    init_w = get_model_weights(model)
    rng = np.random.RandomState(777)
    for _ in range(N):
        v = rng.normal(size=(len(init_w,)), scale=sigma)
        set_model_weights(model, init_w + v)
        reward = L(w=init_w + v, loss=loss, x_tr=x_tr, y_tr=y_tr)
        grad += v * reward
    set_model_weights(model, init_w)
    return grad / (N * sigma**2)

In [127]:
def ES_step(model, loss, x_tr, y_tr, lr):
    """
    Params
    ------
    model: torch.nn.Model
        Model to take optimization step on
    loss: torch.nn.Module
        Loss function to optimize
    x_tr: torch.Tensor, (n_examples, n_features)
        Batch to compute gradient over
    y_tr: torch.Tensor, (n_examples, )
        Batch to compute gradient over
    lr: float
        Learning rate to use
    """
    grad = ES_grad(model=model, loss=loss, x_tr=x_tr, y_tr=y_tr, sigma=)
    set_model_weights()

In [147]:
ES_grad(model, loss, x_tr=x_train, y_tr=y_train, sigma=0.001, N=10)

array([-251.85761036, -424.38760101, 1255.30842047, ..., -592.70242856,
        652.9320004 , 1009.17007192])

In [130]:
model = build_model(784, 10)
loss = torch.nn.CrossEntropyLoss(size_average=True)
init_w = get_model_weights(model)

2.3300431823730468

In [51]:
z = list(model.parameters())

In [52]:
z[1]

Parameter containing:
1.00000e-02 *
 0.9429  3.2923  3.1312  ...   1.3594 -0.2681  3.9460
 3.1556 -0.4204  2.5779  ...  -4.0802 -0.2860 -0.3668
-1.9587  0.5703 -4.1915  ...  -3.7700  0.6600 -3.0861
          ...             ⋱             ...          
 4.0039  2.2218  1.1179  ...   2.5937 -0.4478 -0.9402
-0.8708 -2.9625  1.9891  ...  -2.6581  0.0744 -3.0522
-1.5492  2.2387 -0.6655  ...  -1.5889  0.7705 -4.3341
[torch.FloatTensor of size 10x512]

In [ ]:
def sgd_step(model, loss, x_val, y_val, lr, momentum):
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    x = Variable(x_val, requires_grad=False)
    y = Variable(y_val, requires_grad=False)

    # Reset gradient
    optimizer.zero_grad()

    # Forward
    fx = model.forward(x)
    output = loss.forward(fx, y)

    # Backward
    output.backward()

    # Update parameters
    optimizer.step()

    return output.data[0]

## Implement own module

PyTorch enable defining own Modules. Here we will implement Linear and ReQu modules ourselves, to understand better how backpropagation works. 

Below you can find an example Module implementing the classial ReLU nonlinearity.

Ref: https://github.com/jcjohnson/pytorch-examples

In [154]:
class MyReLU(torch.autograd.Function):
  """
  We can implement our own custom autograd Functions by subclassing
  torch.autograd.Function and implementing the forward and backward passes
  which operate on Tensors.
  """
  def forward(self, input):
    """
    In the forward pass we receive a Tensor containing the input and return a
    Tensor containing the output. You can cache arbitrary Tensors for use in the
    backward pass using the save_for_backward method.
    """
    self.save_for_backward(input)
    return input.clamp(min=0)

  def backward(self, grad_output):
    """
    In the backward pass we receive a Tensor containing the gradient of the loss
    with respect to the output, and we need to compute the gradient of the loss
    with respect to the input.
    """
    input, = self.saved_tensors
    grad_input = grad_output.clone()
    grad_input[input < 0] = 0
    return grad_input

### Implement Linear module

### Implement “ReQU” unit

<p>
<font size=4>
Here, we’ll implement a made-up activation function that we’ll call the Rectified Quadratic Unit
(ReQU). Like the sigmoid and ReLU and several others, it is applied element-wise to all its
inputs:
</font>
</p>

<p>
<font size=4>
$$z_i = I[x_i > 0] x_i ^ 2$$
</font>
</p>

(Note, exercise is taken from  https://www.cs.ox.ac.uk/people/nando.defreitas/machinelearning/practicals/practical4.pdf)

In [153]:
model = build_model(n_features, n_classes, nonlinearity_module=MyReLU)
loss = torch.nn.CrossEntropyLoss(size_average=True)
train(step_fnc=sgd_step, loss=loss, model=model)

TypeError: __main__.MyReLU is not a Module subclass

## Implement backpropagation


### Implement custom activation function

### Implement backward traversal

# Tests